# Advanced Visualisation of Lorenz Attractor
---

***Author: Piotr Skalski***

This notebook is a simple tutorial about creating advanced, animated visualizations using Matplotlib. I will show you step by step how to build a fantastic effect with the help of simple tools.

### Imports

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import matplotlib.collections as mcoll
import matplotlib.path as mpath

### Settings

In [97]:
# length of the time step
DELTA_T = 0.01
# number of time steps
STEPS = 5000
# plot padding
PADDING = 2.5
# output directory (the folder must be created on the drive)
OUTPUT_DIR = "lorenz_attractor"

### Auxiliary functions

In [98]:
def lorenz(x, y, z, s=10, r=28, b=2.667):
    x_dot = s*(y - x)
    y_dot = r*x - y - x*z
    z_dot = x*y - b*z
    return x_dot, y_dot, z_dot

def build_lorenz_trajectory(dt, steps, initial_values=(0., 1., 1.05), s=10, r=28, b=2.667):
    xs = np.empty((steps + 1,))
    ys = np.empty((steps + 1,))
    zs = np.empty((steps + 1,))
    
    xs[0], ys[0], zs[0] = initial_values
    
    for i in range(steps):
        x_dot, y_dot, z_dot = lorenz(xs[i], ys[i], zs[i])
        xs[i + 1] = xs[i] + (x_dot * dt)
        ys[i + 1] = ys[i] + (y_dot * dt)
        zs[i + 1] = zs[i] + (z_dot * dt)
        
    return (xs, ys, zs)

### Create trajectory data

In [99]:
xs, ys, zs = build_lorenz_trajectory(DELTA_T, STEPS)

### Render frames

In [ ]:
padding = PADDING

plt.style.use('dark_background')
fig = plt.figure(figsize=(12,12))
ax = fig.gca(projection='3d')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

ax.set_xlim3d(xs.min() - padding, xs.max() + padding)
ax.set_ylim3d(ys.min() - padding, ys.max() + padding)
ax.set_zlim3d(zs.min() - padding, zs.max() + padding)

ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")

N = len(zs)
# ax.scatter(xs, ys, zs, c = plt.cm.jet(np.linspace(0,1,N)), alpha=0.5)

for i in range(N-1):
    ax.plot(xs[i:i+2], ys[i:i+2], zs[i:i+2], color=plt.cm.viridis(1-i/N), lw=2)
    if(i % 5 == 0):
        angle_delta = i/5
        ax.view_init(30, 45 + angle_delta)
        plt.savefig("./" + OUTPUT_DIR + "/lorenz_{:0>5d}.png".format(i))

plt.show()